In [1]:
%matplotlib inline
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import logm

In [2]:
## parameters, UD=underdamped, OD=overdamped
N   = 2**20            # # of data points
m   = 1                # mass
T   = 1                # temperature
k   = 1                # stiffness
k_B = 1                # Boltzmann constant
f_s = 1                # sampling frequency
dt  = 1/f_s            # step size
g_c = np.sqrt(4*m*k)   # gamma critical
g_f = .5               # <1 for UD and >1 for OD
g   = g_c*g_f          # gamma
w0  = np.sqrt(k/m)     # frequency of UD motion
tau = m/g              # relaxation time
D   = k_B*T/g          # diffusion constant

if g_f >1:
    w = 1j*np.sqrt(-(w0**2)+(1/(4*tau*tau))) 
else:
    w = np.sqrt((w0**2)-(1/(4*tau*tau)))

params = (N, dt, w, w0, tau)

In [3]:
def calcSigma(params):
    N, dt, w, w0, tau = params
    tt   = 2*w*dt;  w02  = w0*w0;  w2 = w*w
    tau2 = tau*tau; tau3 = tau2*tau; dtbt=-dt/tau
    wt2=w2*tau2; dd=D/(wt2); dd1=dd/(4*w02*tau); dd2=dd/(4*tau)

    ss1 = np.cos(tt) - (2*w*tau*np.sin(tt)) - (4*w02*tau2)
    ss2 = np.cos(tt) + (2*w*tau*np.sin(tt)) - (4*w02*tau2)

    sigma2_xx = dd1*(4*w2*tau2 + np.exp(dtbt)*ss1)
    sigma2_vv = dd2*(4*w2*tau2 + np.exp(dtbt)*ss2)
    sigma2_xv = dd*np.exp(dtbt)*np.sin(w*dt)*np.sin(w*dt)
    return np.real(sigma2_xx), np.real(sigma2_vv), np.real(sigma2_xv)    


def calcLambda(params):
    N, dt, w, w0, tau = params
    ii  = np.eye(2)
    ll = np.asanyarray([[0, -1], [k/m, g/m]])
    ee = np.exp(-dt/(2*tau)); wt2=2*w*tau
    cc=np.cos(w*dt); ss=np.sin(w*dt) 
    
    Lambda = ee*((cc+ss/wt2)*ii - ll*ss/w ) 
    return np.real(Lambda)


def calcXV(params, Lambda, Sigma):
    N, dt, w, w0, tau = params
    x = np.zeros([N,1])
    v = np.zeros([N,1])
    
    sigma2_xx, sigma2_vv, sigma2_xv = Sigma

    for j in np.arange(0,N-1):
        oldvec = np.array([x[j],v[j]])
        randgauss = np.random.randn(2,1)
        delx = np.sqrt(sigma2_xx)*randgauss[0]
        delv = (sigma2_xv/(np.sqrt(sigma2_xx)))*randgauss[0]+(np.sqrt(sigma2_vv - ((sigma2_xv**2)/(sigma2_xx))))*randgauss[1]
        delvec = np.array([delx,delv])
        updatevec = np.dot(Lambda,oldvec)+delvec
        x[j+1] = updatevec[0]
        v[j+1] = updatevec[1]
    return x,v

In [4]:
params = (N, dt, w, w0, tau)
Lambda = calcLambda(params)
Sigma  = calcSigma(params)
x, v   = calcXV(params, Lambda, Sigma)

In [5]:
#plt.plot(x);plt.plot(v);

## Bayes I

In [6]:
# matrix sufficient statistics
T1_11 = np.sum(x[1:]**2)
T1_12 = np.sum(x[1:]*v[1:])
T1_21 = T1_12
T1_22 = np.sum(v[1:]**2)

T2_11 = np.sum(x[1:]*x[:-1])
T2_12 = np.sum(x[1:]*v[:-1])
T2_21 = np.sum(v[1:]*x[:-1])
T2_22 = np.sum(v[1:]*v[:-1])

T3_11 = np.sum(x[:-1]*x[:-1])
T3_12 = np.sum(x[:-1]*v[:-1])
T3_21 = T3_12
T3_22 = np.sum(v[:-1]*v[:-1])

T1    = np.asanyarray([[T1_11, T1_12],[T1_21, T1_22]])
T2    = np.asanyarray([[T2_11, T2_12],[T2_21, T2_22]])
T3    = np.asanyarray([[T3_11, T3_12],[T3_21, T3_22]])

In [7]:
#MAP estimate of Lambda and Sigma

invT3     = np.linalg.inv(T3)
LambdaMAP = np.dot(T2, invT3)
SigmaMAP  = (1/N)*( T1 - np.dot( T2, np.dot(invT3, T2.transpose()) ) )

In [8]:
#MAP estimate of c using Onsager-Casimir symmetry

L_t  = LambdaMAP.transpose()
eps  = np.asanyarray([[1,0],[0,-1]])

L_te = np.dot(eps, L_t)
Lte2 = np.dot(L_te, L_te)

II   = np.eye(2)
cc   = np.linalg.inv(II - Lte2)
cMAP = np.dot(SigmaMAP, cc)

print LambdaMAP, '\n', SigmaMAP, '\n', cMAP

[[ 0.65963655  0.53432075]
 [-0.53361729  0.12754866]] 
[[ 0.28028526  0.28459618]
 [ 0.28459618  0.69973405]] 
[[  1.00157031e+00   3.13499929e-04]
 [ -1.01981697e-03   1.00202555e+00]]


In [9]:
#MAP estimate of k and m and gamma

ll = -logm(LambdaMAP)/dt

m_MAP = k_B*T/cMAP[1,1]
k_MAP = k_B*T/cMAP[0,0]
g_MAP = m*ll[1,1]


print m_MAP, k_MAP, g_MAP

0.997978545612 0.998432147188 0.995896233509


## Bayes II

In [10]:
#MAP estimate of c using Bayes II
c_B2 = T3/N
print c_B2

[[  1.00235126e+00   2.01967007e-05]
 [  2.01967007e-05   1.00143982e+00]]
